In [6]:
import Pkg; 

# Add packages
Pkg.add("RigidBodyDynamics"); 
Pkg.add("LinearAlgebra"); 
Pkg.add("StaticArrays");

# Use packages
using RigidBodyDynamics; 
using LinearAlgebra; 
using StaticArrays; 

  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]


In [7]:
g = -9.81
world = RigidBody{Float64}("world")
doublependulum = Mechanism(world, gravity=SVector(0, 0, g))

Spanning tree:
Vertex: world (root)
No non-tree joints.

## Create double link pendulum 

In [10]:
axis = SVector(0., 1., 0) # joint axis
I_1 = 0.333
c_1 = -0.5
m_1 = 1
frame1 = CartesianFrame3D("upper_link")
inertia1 = SpatialInertia(frame1, 
            moment=I_1 * axis * axis', 
            com=SVector(0, 0, c_1), 
            mass=m_1)

SpatialInertia expressed in "upper_link" (id = 3):
mass: 1.0
center of mass: Point3D in "upper_link": [0.0, 0.0, -0.5]
moment of inertia (about origin of "upper_link" (id = 3):
[0.0 0.0 0.0; 0.0 0.333 0.0; 0.0 0.0 0.0]

In [17]:
upperlink = RigidBody(inertia1)

RigidBody: "upper_link"

In [18]:
shoulder = Joint("shoulder", Revolute(axis))

Joint "shoulder": Revolute joint with axis [0.0, 1.0, 0.0]

In [19]:
before_shoulder_to_world = one(Transform3D, 
                           frame_before(shoulder), default_frame(world))

Transform3D from "before_shoulder" to "world":
rotation: 0.0 rad about [1.0, 0.0, 0.0], translation: [0.0, 0.0, 0.0]

In [21]:
attach!(doublependulum, world, upperlink, shoulder,
        joint_pose = before_shoulder_to_world)

Spanning tree:
Vertex: world (root)
  Vertex: upper_link, Edge: shoulder
No non-tree joints.

In [22]:
l_1 = -1; 
I_2 = 0.333; 
c_2 = -0.5; 
m_2 = 1.; 

inertia2 = SpatialInertia(CartesianFrame3D("lower_link"), 
            moment=I_2 * axis * axis', 
            com=SVector(0, 0, c_2), 
            mass=m_2)
lowerlink = RigidBody(inertia2)
elbow = Joint("elbow", Revolute(axis))
before_elbow_to_after_shoulder = Transform3D(
    frame_before(elbow), frame_after(shoulder), SVector(0, 0, l_1))
attach!(doublependulum, upperlink, lowerlink, elbow,
    joint_pose = before_elbow_to_after_shoulder)

Spanning tree:
Vertex: world (root)
  Vertex: upper_link, Edge: shoulder
    Vertex: lower_link, Edge: elbow
No non-tree joints.

In [23]:
state = MechanismState(doublependulum)

MechanismState{Float64, Float64, Float64, …}(…)

In [24]:
set_configuration!(state, shoulder, 0.3);
set_configuration!(state, elbow, 0.4);
set_velocity!(state, shoulder, 1.);
set_velocity!(state, elbow, 2.);


In [25]:
q = configuration(state)
v = velocity(state)

2-element SegmentedVector{JointID,Float64,Base.OneTo{JointID},Array{Float64,1}}:
 0.0
 0.0

In [27]:
transform(state, Point3D(frame_after(elbow), zero(SVector{3})), 
        default_frame(world))

Point3D in "world": [-0.29552020666133955, 0.0, -0.955336489125606]

In [28]:
transform_to_root(state, frame_after(elbow))

Transform3D from "after_elbow" to "world":
rotation: 0.3 rad about [1.6266700491214728e-62, 1.0, 0.0], translation: [-0.29552020666133955, 0.0, -0.955336489125606]

In [29]:
relative_transform(state, frame_after(elbow), frame_after(shoulder))

Transform3D from "after_elbow" to "after_shoulder":
rotation: 0.0 rad about [1.0, 0.0, 0.0], translation: [9.370825333944079e-18, 0.0, -0.9999999999999999]

In [30]:
center_of_mass(state)

Point3D in "world": [-0.29552020666133955, 0.0, -0.955336489125606]

In [31]:
mass_matrix(state)

2×2 Symmetric{Float64,Array{Float64,2}}:
 2.666  0.833
 0.833  0.333

**useful info:** the `!` at the end of a method is a Julia convention signifying that the function is 'in-place', i.e. modifies its input data).

In [32]:
v = similar(velocity(state))
v[shoulder][1] = 1; 
v[elbow][1] = 2; 
inverse_dynamics(state, v)

2-element SegmentedVector{JointID,Float64,Base.OneTo{JointID},Array{Float64,1}}:
 10.130106454695483
  2.948526613673872

In [33]:
ts, qs, vs = simulate(state, 5., Δt = 1e-3)

([0.0, 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009000000000000001  …  4.991000000000001, 4.992000000000002, 4.993000000000002, 4.994000000000002, 4.995000000000003, 4.996000000000003, 4.997000000000003, 4.998000000000004, 4.999000000000004, 5.000000000000004], SegmentedVector{JointID,Float64,Base.OneTo{JointID},Array{Float64,1}}[[0.3, 0.0], [0.29999813472636105, 2.4895176895441913e-6], [0.2999925389580227, 9.95793044882661e-6], [0.2999832128527186, 2.240481735426898e-5], [0.2999701566733337, 3.982947687952129e-5], [0.2999533707878971, 6.223092691488781e-5], [0.2999328556695723, 8.960790479453432e-5], [0.299908611896645, 0.00012195886733148753], [0.2998806401525076, 0.00015928199086044035], [0.299848941225641, 0.00020157517128838013]  …  [0.18550788378260852, 0.1174340494638931], [0.18522041147134746, 0.11664513582384438], [0.18493216193826076, 0.11585376482264699], [0.18464312954071993, 0.11505996583140063], [0.18435330859267104, 0.11426376838142359], [0.1840626933649

In [37]:
Pkg.add("MeshCatMechanisms"); 
Pkg.add("MeshCat")

# Use packages
using MeshCat
using RigidBodyDynamics
using MeshCatMechanisms

  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
  [283c5d60] + MeshCat v0.13.2
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]
┌ Info: Precompiling MeshCat [283c5d60-a78f-5afe-a0af-af636b173e11]
└ @ Base loading.jl:1260
  likely near /home/zozan/.julia/packages/MeshCat/GlCMx/src/MeshCat.jl:31
  likely near /home/zozan/.julia/packages/MeshCat/GlCMx/src/atframe.jl:33
┌ Info: Precompiling MeshCatMechanisms [6ad125db-dd91-5488-b820-c1df6aab299d]
└ @ Base loading.jl:1260


WebIO._IJuliaInit()

### ATLAS paper

[Planning for atlas](https://link.springer.com/content/pdf/10.1007/s10514-015-9479-3.pdf)

In [39]:
URDF.write_urdf("double_pendulum", doublependulum)

## Plotting example

Plotting example can be found [here](https://github.com/JuliaRobotics/MeshCatMechanisms.jl/blob/master/examples/demo.ipynb). 